## Prepping (joining) datasets for DB

In [1]:
# read in cleaned datasets for joins
import pandas as pd
import os

samples_check = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/samples.csv")
morpho = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/morpho.csv")
lab_analysis = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/analyses.csv")
site_info = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/site_info.csv")
soil_type = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/soil_type.csv")
climate = pd.read_csv("//Users/inesschwartz/Desktop/cleandata/climate_feat.csv")
topo = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/topo_feat.csv")
geo = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/geo_feat.csv")
geo_mapping = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/geology_mapping.csv")
litho_mapping = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/lithology_mapping.csv")
litho54_mapping = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/lithology1954_mapping.csv")

#profile_record = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/profile_record_clean.csv")

## checking csv datatypes before importing to db ##

In [2]:
import pandas as pd
import os

# Set the path to your folder containing the 10 CSV files
csv_folder = "/Users/inesschwartz/Desktop/cleandata"  

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Loop through each file and display column data types
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    print(f"\n📄 File: {file}")
    
    try:
        df = pd.read_csv(file_path)
        print(df.dtypes)
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")


📄 File: analyses.csv
lab_sample_id      int64
analysis_id       object
morpho_id         object
sample_id         object
profile           object
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 72, dtype: object

📄 File: soil_type.csv
soil_type_id     int64
profile         object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object

📄 File: geology_mapping.csv
geology_code           object
geology_description    object
dtype: object

📄 File: lithology1954_mapping.csv
lithology_1954_code           object
lithology_1954_description    object
dtype: object

📄 File: morpho.csv
morpho_id                 object
sample_id                float64
profile                   object
horizon_layer            float64
upper_depth              float64
lower_depth              float64
moisture_degree           object
root_quantity             object
root_diameter  

In [5]:
# ✅ STEP 1: Create a mapping of DataFrame objects to the columns to convert
dataframe_column_map = {
    "lab_analysis": (lab_analysis, ["lab_analysis_id", "analysis_id", "sample_id", "profile"]),
    "soil_type": (soil_type, ["soil_type_id"]),
    "site_info": (site_info, ["site_info_id", "profile", "climate_id", "geo_features_id", "topo_id", "soil_type_id"]),
    "topo": (topo, ["topo_id"]),
    "geo": (geo, ["geo_features_id"]),
    "climate": (climate, ["climate_id"])
}

# ✅ STEP 2: Define function to convert specified columns to object type
def convert_columns_to_object(df, columns):
    df = df.copy()
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(
                lambda x: str(int(x)) if pd.notna(x) and isinstance(x, (int, float)) and float(x).is_integer()
                else str(x) if pd.notna(x)
                else pd.NA
            ).astype("object")
            print(f"  🔧 Converted column '{col}' to object.")
    return df

# ✅ STEP 3: Apply transformation to each DataFrame in memory
for name, (df, columns) in dataframe_column_map.items():
    print(f"📂 Processing DataFrame: {name}")
    converted_df = convert_columns_to_object(df, columns)

    # Optionally: overwrite the original DataFrame in memory
    globals()[name] = converted_df

    #Save updated file
    converted_df.to_csv(f"/Users/inesschwartz/Desktop/cleandata/{name}.csv", index=False)
    print(f"✅ Updated in memory: {name}\n")

print("🎯 All specified columns converted to object dtype in memory.")


📂 Processing DataFrame: lab_analysis
  🔧 Converted column 'analysis_id' to object.
  🔧 Converted column 'sample_id' to object.
  🔧 Converted column 'profile' to object.
✅ Updated in memory: lab_analysis

📂 Processing DataFrame: soil_type
  🔧 Converted column 'soil_type_id' to object.
✅ Updated in memory: soil_type

📂 Processing DataFrame: site_info
  🔧 Converted column 'site_info_id' to object.
  🔧 Converted column 'profile' to object.
  🔧 Converted column 'climate_id' to object.
  🔧 Converted column 'geo_features_id' to object.
  🔧 Converted column 'topo_id' to object.
  🔧 Converted column 'soil_type_id' to object.
✅ Updated in memory: site_info

📂 Processing DataFrame: topo
  🔧 Converted column 'topo_id' to object.
✅ Updated in memory: topo

📂 Processing DataFrame: geo
  🔧 Converted column 'geo_features_id' to object.
✅ Updated in memory: geo

📂 Processing DataFrame: climate
  🔧 Converted column 'climate_id' to object.
✅ Updated in memory: climate

🎯 All specified columns converted t

### dataprep work below

In [6]:
# Adding soil_biology_id column (just in case we add a biology table)
lab_analysis['soil_biology_id'] = pd.NA

# Define the final column order
lab_analysis_final_columns = [
    'lab_sample_id', 'analysis_id', 'morpho_id', 'sample_id', 'profile', 'soil_biology_id', 'EG', 'thick_clay', 'fine_clay', 'silt', 'clay', 'Eq_Hum', 'atm_1/3', 'atm_15',
    'CACO3', 'gypsum', 'free_iron', 'organic_carbon', 'total_N', 'P205', 'organic_material', 'pH_H2O', 'pH_KCL',
    'Ca++', 'Mg++', 'Na+', 'K+', 'exchangable_bases_sum', 'CEC', 'V', 'conductivity', 'soluble_sodium', 'Min_<0,002',
    'Min_0,05-0,02', 'Min_0,2-0,05', 'Min_2-0,2', 
    'field_sample_code', 'Depth', 'Al', 'Si', 'P', 'S', 'Cl', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'As', 'Se', 'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Cd', 'Sn', 'Sb', 'Ba', 'Ta', 'W', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Th', 'U'
]

# Filter and save the cleaned DataFrame
lab_analysis[lab_analysis_final_columns].to_csv('/Users/inesschwartz/Desktop/cleandata/analyses.csv', index=False)


## completing site_info mapping table

this is a mapping table to link site info, location to soil samples (which samples and when taken)

In [11]:
site_info.head()

,site_info_id,profile,X_coord,Y_coord,district,geo_features_id,climate_id,topo_id,soil_type_id
0,2770,1_57,12.161278,-15.222598,Namibe,1,1,1,2.0
1,48,1_59,12.575775,-4.866986,Cabinda,2,2,2,3.0
2,1618,1_61,15.098840,-11.225411,Cuanza Sul,3,3,3,NaN
3,881,1_63,17.081955,-9.274587,Malanje,4,4,4,4.0
4,1750,1_64,20.788116,-11.568683,Moxico,5,5,5,NaN


In [12]:
# Convert keys to string before merge to avoid dtype mismatch errors
site_info['site_info_id'] = site_info['ID'].astype(str)
geo['ID'] = geo['ID'].astype(str)
climate['ID'] = climate['ID'].astype(str)
topo['ID'] = topo['ID'].astype(str)

# Drop old FK columns if they exist
site_info = site_info.drop(
    columns=['geo_features_id', 'climate_id', 'topo_features_id', 'district_id', 'land_cover_id', 'geology_id','topo_feature_id','soil_type_id', 'districts_id' ],
    errors='ignore'
)

KeyError: 'ID'

In [17]:
# --- Step 1: Convert 'ID' columns to string ---
site_info['ID'] = site_info['ID'].astype(str)
geo['ID'] = geo['ID'].astype(str)
climate['ID'] = climate['ID'].astype(str)
topo['ID'] = topo['ID'].astype(str)

# --- Step 2: Merge feature tables into site_info_clean ---
site_info = site_info.merge(
    geo[['ID', 'geo_features_id']],
    on='ID',
    how='left'
)

site_info = site_info.merge(
    climate[['ID', 'climate_id']],
    on='ID',
    how='left'
)

site_info = site_info.merge(
    topo[['ID', 'topo_id']],
    on='ID',
    how='left'
)

# --- Step 3: Drop 'ID' column from all involved tables ---
for table in ['site_info', 'geo', 'climate', 'topo']:
    df = globals().get(table)
    if df is not None and 'ID' in df.columns:
        df = df.drop(columns=['ID'])
        globals()[table] = df
        print(f"✅ Dropped 'ID' from {table}")
    else:
        print(f"⏭️ Skipping {table}: not found or no 'ID' column")

# --- District: Skip merging district_id, use district name directly ---

# don't see need for district_id, just use district name
# For district (assuming district columns are already strings)
# site_info_clean = site_info_clean.merge(
#     district_clean[['district', 'district_id']],
#     on='district',
#     how='left'
# )


✅ Dropped 'ID' from site_info
✅ Dropped 'ID' from geo
✅ Dropped 'ID' from climate
✅ Dropped 'ID' from topo


In [31]:
#save updated csvs
geo.to_csv('/Users/inesschwartz/Desktop/cleandata/geo_feat.csv', index=False)
climate.to_csv('/Users/inesschwartz/Desktop/cleandata/climate_feat.csv', index = False)
topo.to_csv('/Users/inesschwartz/Desktop/cleandata/topo_feat.csv', index = False)

In [13]:
# Step 1: Ensure 'profile' columns are strings
site_info['profile'] = site_info['profile'].astype(str)
soil_type['profile'] = soil_type['profile'].astype(str)

# Step 2: Merge 'soil_type_id' from soil_type into site_info via profile
site_info = site_info.merge(
    soil_type[['profile', 'soil_type_id']],
    on='profile',
    how='left'
)


In [14]:
soil_type.dtypes

soil_type_id     int64
profile         object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object

In [15]:
site_info.dtypes

site_info_id         int64
profile             object
X_coord            float64
Y_coord            float64
district            object
geo_features_id      int64
climate_id           int64
topo_id              int64
soil_type_id_x     float64
soil_type_id_y     float64
dtype: object

In [16]:
# Columns to convert to string
id_columns = site_info[["geo_features_id", "climate_id", "topo_id", "soil_type_id"]]

# Force conversion to string (robust handling of int, float, NaN)
for col in id_columns:
    if col in site_info.columns:
        site_info[col] = site_info[col].apply(
            lambda x: str(int(x)) if pd.notna(x) and isinstance(x, (float, int)) and float(x).is_integer()
            else str(x) if pd.notna(x)
            else pd.NA
        ).astype("string")
        print(f"✅ Converted {col} to string.")

# Optional: verify the result
print("\n📊 Data types after conversion:")
print(site_info.dtypes)

# # Save the updated CSV
# site_info.to_csv(site_info_path, index=False)
# print("\n💾 site_info.csv saved with updated string columns.")

KeyError: "['soil_type_id'] not in index"

In [37]:
#save to csv
site_info.to_csv('/Users/inesschwartz/Desktop/cleandata/site_info.csv', index = False)

## Completing samples table

In [25]:
samples_check.head()

,sample_id,profile,year,shelf,room,site_info_id
0,630,139,1946.0,1,22,NaN
1,631,139,1946.0,1,22,NaN
2,632,139,1946.0,1,22,NaN
3,633,139,1946.0,1,22,NaN
4,687,208,1946.0,1,22,NaN


In [23]:
# Drop existing site_info_id, morpho_id, and lab_info_id columns if present
samples_check = samples_check.drop(columns=['site_info_id', 'morpho_id', 'lab_info_id'], errors='ignore')

# Merge site_info_id into samples_check using profile
samples_check = samples_check.merge(
    site_info[['site_info_id', 'profile']],
    on='profile',
    how='left'
)


In [24]:
# 1. Check how many rows in samples_check are missing site_info_id
missing_count = samples_check['site_info_id'].isna().sum()
print(f"Number of unmatched profiles (missing site_info_id): {missing_count}")

# 2. Show the unmatched profiles
unmatched_profiles = samples_check[samples_check['site_info_id'].isna()]['profile'].unique()
print("Profiles in samples_check with no match in site_info:")
print(unmatched_profiles)

# Optional: count how many times each unmatched profile appears
unmatched_counts = samples_check[samples_check['site_info_id'].isna()]['profile'].value_counts()
print("\nFrequency of unmatched profiles:")
print(unmatched_counts)


Number of unmatched profiles (missing site_info_id): 4070
Profiles in samples_check with no match in site_info:
['139' '208' '227' '238' '245' '252' '11' '6' '234' '221' '240' '1' '241'
 '280' '21-C' '16-C' '26-C' '165-C' '167-C' '262-C' '139-C' '128-C'
 '382-C' '526-C' '490-C' '492-C' '217-C' '192-C' '286-C' '420-C' '219-C'
 '462-C' '460-C' '39-C' '31-C' '207-C' '303-C' '1092' '1074' '1106' '1016'
 '1152' '1035' '1068' '1504' '1253' '1231' '1278' '1249' '1179' '1286'
 '1306' '1365' '1372' '1313' '1299' '1554' '1637' '1611' '1353' '1280'
 '378' '1364' '2055' '2391' '2215' '5' '4' '9' '10' '15' '17' '18' '19'
 '28' '24' '31' '20' '37' '33' '42' '48' '49' '53' '52' '57' '56' '62'
 '61' '65' '67' '85' '93' '95' '96' '98' '100' '103' '102' '101' '104'
 '113' '110' '111' '114' '118' '121' '122' '130' '138' '159' '162' '172'
 '183' '189' '214' '224' '228' '231' '243' '259' '286' '296' '298' '308'
 '303' '325' '336' '333' '337' '340' '348' '347' '362' '367' '370' '382'
 '392' '397' '400' '406

In [26]:
samples_check.to_csv('/Users/inesschwartz/Desktop/cleandata/samples.csv', index = False)